
# 🚗 Cars Dataset — Linear Regression (Step-by-Step, Beginner Friendly)
*Generated on 2025-09-12 03:29:55*

This notebook teaches **Linear Regression** with a Cars dataset in **small, easy steps**.  
Each step ends with a short **📝 TODO** so you can practice.

What you'll do:
1) Download + load the dataset (via **gdown**)  
2) Inspect + clean data (simple)  
3) Univariate plots (histograms)  
4) Single-variable Linear Regression (fit + plot)  
5) Multi-variable Linear Regression (fit + metrics)  
6) Diagnostic plots (residuals)  
7) Polynomial features + **GridSearchCV** to find the **best degree**  
8) Brief coefficient interpretation


## 0) Setup

In [1]:

#@title Install & import (safe to re-run)
import sys, subprocess

def pip_install(pkg):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=False)

pip_install("gdown>=5.1")
pip_install("pandas>=1.5")
pip_install("numpy>=1.23")
pip_install("matplotlib>=3.7")
pip_install("scikit-learn>=1.3")

import gdown, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


## 1) Download & Load the Dataset

In [ ]:

#@title Download CSV via gdown
GDOWN_ID = "1bwRmKkPwmLKiqOgQ_LnKH0Vsc3mJKmVR"  # provided ID
OUTPUT_CSV = "cars.csv"

if not os.path.exists(OUTPUT_CSV):
    url = f"https://drive.google.com/uc?id={GDOWN_ID}"
    gdown.download(url, OUTPUT_CSV, quiet=False)
else:
    print("Found existing file:", OUTPUT_CSV)

df = pd.read_csv(OUTPUT_CSV)
print("✅ Loaded:", df.shape)
df.head()



### 📝 TODO
- Skim the first few rows above. What looks like a good **target** (y) for prediction (e.g., `mpg`, `price`)?


In [2]:
TARGET = "mpg"

if TARGET not in df.columns:
    numeric_cols_all = df.select_dtypes(include=np.number).columns.tolist()
    if numeric_cols_all:
        TARGET = numeric_cols_all[0]
        print("Auto-selected TARGET =", TARGET)

numeric_cols = [c for c in df.select_dtypes(include=np.number).columns if c != TARGET]

print("Target:", TARGET)
print("Numeric features (first 6 shown):", numeric_cols[:6])


## 2) Quick Inspect & Simple Cleaning

In [ ]:

print("Columns:", list(df.columns))
print("\nData types:\n", df.dtypes)
print("\nMissing values per column:\n", df.isna().sum())

# Simple cleaning: drop duplicates
before = df.shape[0]
df = df.drop_duplicates().reset_index(drop=True)
print("\nDropped duplicates:", before - df.shape[0])

# (Optional) strip/underscore column names
df.columns = [c.strip().replace(" ", "_") for c in df.columns]



### 📝 TODO
- If you see obvious bad rows (e.g., impossible negative values), write one **extra line** to filter them out.


In [ ]:
print(f"Before cleaning: {df.shape[0]} rows")
positive_cols = ['mpg', 'horsepower', 'weight', 'displacement']
for col in positive_cols:
    if col in df.columns:
        bad_rows = df[col] <= 0
        if bad_rows.any():
            print(f"Removing {bad_rows.sum()} rows with negative {col}")
            df = df[~bad_rows]
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col in df.columns:
        mean_val = df[col].mean()
        std_val = df[col].std()
        outliers = (df[col] < mean_val - 3 * std_val) | (df[col] > mean_val + 3 * std_val)
        if outliers.any():
            print(f"Removing {outliers.sum()} extreme outliers from {col}")
            df = df[~outliers]
print(f"After cleaning: {df.shape[0]} rows")
df = df.reset_index(drop=True)

## 3) Choose Target and Features (Keep it Simple)

In [ ]:

# 👉 Set your target (change this if needed)
TARGET = "mpg"  # <-- change if your dataset uses a different target

# If the target isn't present, try to auto-pick a numeric column
if TARGET not in df.columns:
    numeric_cols_all = df.select_dtypes(include=np.number).columns.tolist()
    if numeric_cols_all:
        TARGET = numeric_cols_all[0]
        print("Auto-selected TARGET =", TARGET)

# We'll work **only with numeric predictors** for simplicity.
numeric_cols = [c for c in df.select_dtypes(include=np.number).columns if c != TARGET]

# Quick sanity
print("Target:", TARGET)
print("Numeric features (first 6 shown):", numeric_cols[:6])



### 📝 TODO
- If you prefer different features, create a **manual list**, e.g.  
  `numeric_cols = ['horsepower','weight','displacement','acceleration']` (only if those exist).


In [ ]:
if TARGET in df.columns:
    correlations = df[numeric_cols + [TARGET]].corr()[TARGET].abs().sort_values(ascending=False)
    print(f"Feature correlations with {TARGET}:")
    print(correlations)
    top_features = correlations.drop(TARGET).head(3).index.tolist()
    print(f"\nTop 3 most correlated features: {top_features}")
    numeric_cols = top_features
    print(f"Using features: {numeric_cols}")
else:
    print("Target not found, using all numeric features")

## 4) Univariate Plots (Matplotlib)

In [ ]:

# Plot target histogram
if TARGET in df.columns and pd.api.types.is_numeric_dtype(df[TARGET]):
    plt.figure()
    df[TARGET].plot(kind='hist', bins=30, title=f"Histogram of {TARGET}")
    plt.xlabel(TARGET); plt.ylabel("Frequency")
    plt.show()

# Plot up to 3 numeric features' histograms
for col in numeric_cols[:3]:
    plt.figure()
    df[col].plot(kind='hist', bins=30, title=f"Histogram of {col}")
    plt.xlabel(col); plt.ylabel("Frequency")
    plt.show()



### 📝 TODO
- Looking at the histograms, note any skewed variables or outliers.


In [ ]:
print("Distribution Analysis:")
print("=" * 30)

for col in [TARGET] + numeric_cols:
    if col in df.columns:
        data = df[col].dropna()
        skewness = data.skew()
        print(f"{col}:")
        print(f"  Skewness: {skewness:.2f}")
        if abs(skewness) > 1:
            print(f"  ⚠️  Heavily skewed")
        elif abs(skewness) > 0.5:
            print(f"  ⚠️  Moderately skewed")
        else:
            print(f"  ✅ Approximately normal")

print("\nNote: Heavily skewed variables may need transformation")

## 5) Single-Variable Linear Regression

In [ ]:

# Pick one feature (auto-pick the first numeric feature)
if len(numeric_cols) == 0:
    raise ValueError("No numeric predictors found. Please adjust `numeric_cols`.")

FEATURE_X = numeric_cols[0]  # change to try others

# Drop rows with missing target/feature
data_1v = df[[FEATURE_X, TARGET]].dropna().copy()

X = data_1v[[FEATURE_X]].values
y = data_1v[TARGET].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predictions on test set
y_pred = lr.predict(X_test)

print("Feature:", FEATURE_X)
print("Coefficient (slope):", lr.coef_[0])
print("Intercept:", lr.intercept_)
print("R^2 (test):", r2_score(y_test, y_pred))

# Plot scatter + regression line (on test set)
plt.figure()
plt.scatter(X_test, y_test, label="Actual")
# create a line
x_line = np.linspace(X.min(), X.max(), 200).reshape(-1,1)
y_line = lr.predict(x_line)
plt.plot(x_line, y_line, label="Fit")
plt.xlabel(FEATURE_X); plt.ylabel(TARGET); plt.title("Single-Variable Linear Regression")
plt.legend()
plt.show()



### 📝 TODO
- Change `FEATURE_X` to a different column and re-run.
- Does the line slope match your intuition about the relationship?


In [ ]:
print("Single Feature Comparison:")
print("=" * 30)

best_r2 = 0
best_feature = None

for feature in numeric_cols:
    if feature == TARGET:
        continue
    data_test = df[[feature, TARGET]].dropna().copy()
    X_test_feat = data_test[[feature]].values
    y_test_feat = data_test[TARGET].values
    X_tr, X_te, y_tr, y_te = train_test_split(X_test_feat, y_test_feat, test_size=0.2, random_state=42)
    lr_test = LinearRegression()
    lr_test.fit(X_tr, y_tr)
    y_pred_test = lr_test.predict(X_te)
    r2_test = r2_score(y_te, y_pred_test)
    print(f"{feature:15s}: R² = {r2_test:.3f}")
    if r2_test > best_r2:
        best_r2 = r2_test
        best_feature = feature

print(f"\n🏆 Best feature: {best_feature} (R² = {best_r2:.3f})")
FEATURE_X = best_feature

## 6) Multi-Variable Linear Regression (Simple)

In [ ]:

# Keep only numeric predictors + target; drop NA
data_mv = df[numeric_cols + [TARGET]].dropna().copy()

X = data_mv[numeric_cols].values
y = data_mv[TARGET].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lin = LinearRegression()
lin.fit(X_train, y_train)

y_pred = lin.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"MAE:  {mae:.3f}")
print(f"MSE:  {mse:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R^2:  {r2:.3f}")



### 📝 TODO
- Remove one weak feature from `numeric_cols` and see how metrics change.
- Add a different feature and compare.


In [ ]:
print("Feature Combination Testing:")
print("=" * 30)

# Original model (all features)
data_orig = df[numeric_cols + [TARGET]].dropna().copy()
X_orig = data_orig[numeric_cols].values
y_orig = data_orig[TARGET].values
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_orig, y_orig, test_size=0.2, random_state=42)
lin_orig = LinearRegression()
lin_orig.fit(X_train_orig, y_train_orig)
y_pred_orig = lin_orig.predict(X_test_orig)
r2_orig = r2_score(y_test_orig, y_pred_orig)
print(f"All features: R² = {r2_orig:.3f}")

# Test with only top 2 features
correlations = df[numeric_cols + [TARGET]].corr()[TARGET].abs().sort_values(ascending=False)
top_2_features = correlations.drop(TARGET).head(2).index.tolist()
data_top2 = df[top_2_features + [TARGET]].dropna().copy()
X_top2 = data_top2[top_2_features].values
y_top2 = data_top2[TARGET].values
X_train_top2, X_test_top2, y_train_top2, y_test_top2 = train_test_split(X_top2, y_top2, test_size=0.2, random_state=42)
lin_top2 = LinearRegression()
lin_top2.fit(X_train_top2, y_train_top2)
y_pred_top2 = lin_top2.predict(X_test_top2)
r2_top2 = r2_score(y_test_top2, y_pred_top2)
print(f"Top 2 features: R² = {r2_top2:.3f}")

# Summary
print(f"\nBest model: {'Top 2 features' if r2_top2 > r2_orig else 'All features'}")
if r2_top2 > r2_orig:
    numeric_cols = top_2_features

## 7) Diagnostic Plots (Residuals)

In [ ]:

# Reuse y_test and y_pred from multi-variable model
residuals = y_test - y_pred

# Predicted vs Actual
plt.figure()
plt.scatter(y_test, y_pred)
plt.xlabel("Actual"); plt.ylabel("Predicted")
plt.title("Predicted vs Actual (Test)")
plt.show()

# Residuals vs Predicted
plt.figure()
plt.scatter(y_pred, residuals)
plt.axhline(0, linestyle='--')
plt.xlabel("Predicted"); plt.ylabel("Residuals")
plt.title("Residuals vs Predicted")
plt.show()

# Histogram of residuals
plt.figure()
pd.Series(residuals).plot(kind='hist', bins=30, title="Histogram of Residuals")
plt.xlabel("Residual"); plt.ylabel("Frequency")
plt.show()



### 📝 TODO
- Do residuals look roughly centered around 0 and evenly spread?
- If not, which assumption might be violated?


In [ ]:
print("Residual Analysis:")
print("=" * 25)

# Calculate residuals
residuals = y_test - y_pred

# Basic checks
print(f"Mean of residuals: {residuals.mean():.4f} (should be ≈ 0)")
print(f"Residuals centered: {'✅ YES' if abs(residuals.mean()) < 0.1 else '❌ NO'}")

# Visual checks
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Residuals vs Predicted
axes[0].scatter(y_pred, residuals, alpha=0.6)
axes[0].axhline(0, color='red', linestyle='--')
axes[0].set_xlabel('Predicted Values')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residuals vs Predicted')
axes[0].grid(True, alpha=0.3)

# Histogram of residuals
axes[1].hist(residuals, bins=15, alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Residuals')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Simple normality check
from scipy import stats
shapiro_stat, shapiro_p = stats.shapiro(residuals)
print(f"Normality test p-value: {shapiro_p:.4f}")
print(f"Residuals normal: {'✅ YES' if shapiro_p > 0.05 else '❌ NO'}")

print("\nNote: If assumptions violated, consider transformations")

## 8) Polynomial Features + Grid Search for Best Degree


Even though it's called "polynomial regression", it's still **linear regression** applied to **polynomially-expanded features**.
We'll try degrees **1 to 5** on a **single predictor** and pick the degree with the best cross-validated **R²**.


In [ ]:

# Choose a single predictor again (same FEATURE_X as before by default)
FEATURE_X = FEATURE_X  # keep same, or set to something else like 'horsepower'

poly_data = df[[FEATURE_X, TARGET]].dropna().copy()
X = poly_data[[FEATURE_X]].values
y = poly_data[TARGET].values

pipe = Pipeline([
    ("poly", PolynomialFeatures(include_bias=False)),
    ("lr", LinearRegression())
])

param_grid = {
    "poly__degree": [1, 2, 3, 4, 5],
    "lr__fit_intercept": [True, False]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring="r2", n_jobs=-1)
grid.fit(X, y)

print("Best params:", grid.best_params_)
print("Best CV R^2:", grid.best_score_)

best_model = grid.best_estimator_

# Plot best curve
x_line = np.linspace(X.min(), X.max(), 200).reshape(-1,1)
y_line = best_model.predict(x_line)

plt.figure()
plt.scatter(X, y, s=12)
plt.plot(x_line, y_line)
plt.xlabel(FEATURE_X); plt.ylabel(TARGET)
plt.title("Best Polynomial Fit (via Grid Search)")
plt.show()



### 📝 TODO
- Change the degree range (e.g., 1–8) and re-run. Does performance keep improving?
- Try a different `FEATURE_X`.


In [ ]:
print("Polynomial Degree Testing:")
print("=" * 30)

# Use the best single feature
poly_data = df[[FEATURE_X, TARGET]].dropna().copy()
X_poly = poly_data[[FEATURE_X]].values
y_poly = poly_data[TARGET].values

# Test degrees 1, 2, 3
degrees = [1, 2, 3]
results = []

for degree in degrees:
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly_transformed = poly.fit_transform(X_poly)

    # Train-test split
    X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(
        X_poly_transformed, y_poly, test_size=0.2, random_state=42)

    # Fit model
    lr_poly = LinearRegression()
    lr_poly.fit(X_train_poly, y_train_poly)

    # Predictions
    y_pred_poly = lr_poly.predict(X_test_poly)
    r2_poly = r2_score(y_test_poly, y_pred_poly)

    results.append((degree, r2_poly))
    print(f"Degree {degree}: R² = {r2_poly:.3f}")

# Find best degree
best_degree, best_r2 = max(results, key=lambda x: x[1])
print(f"\n🏆 Best degree: {best_degree} (R² = {best_r2:.3f})")

# Plot the best polynomial fit
poly_best = PolynomialFeatures(degree=best_degree, include_bias=False)
X_poly_best = poly_best.fit_transform(X_poly)
lr_best = LinearRegression()
lr_best.fit(X_poly_best, y_poly)

# Create smooth line for plotting
x_line = np.linspace(X_poly.min(), X_poly.max(), 200).reshape(-1, 1)
x_line_poly = poly_best.fit_transform(x_line)
y_line = lr_best.predict(x_line_poly)

plt.figure(figsize=(8, 5))
plt.scatter(X_poly, y_poly, alpha=0.6, label='Data')
plt.plot(x_line, y_line, 'r-', linewidth=2, label=f'Degree {best_degree} fit')
plt.xlabel(FEATURE_X)
plt.ylabel(TARGET)
plt.title(f'Best Polynomial Fit (Degree {best_degree})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 9) Interpreting Coefficients (Multi-Variable Model)

In [ ]:

# Show top coefficients by magnitude (multi-variable linear model)
coef = lin.coef_
coef_df = pd.DataFrame({"feature": numeric_cols, "coefficient": coef})
coef_df["abs_coef"] = coef_df["coefficient"].abs()
coef_df.sort_values("abs_coef", ascending=False, inplace=True)
coef_df.drop(columns=["abs_coef"], inplace=True)
coef_df.head(10)



### 📝 TODO
- Which features have the largest (absolute) coefficients?
- Do the signs (+/-) match your intuition?


In [ ]:
print("Coefficient Analysis:")
print("=" * 25)

# Get coefficients from the multi-variable model
coef = lin.coef_
feature_names = numeric_cols

# Create coefficient dataframe
coef_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": coef,
    "abs_coefficient": np.abs(coef)
})

# Sort by absolute coefficient value
coef_df_sorted = coef_df.sort_values("abs_coefficient", ascending=False).reset_index(drop=True)

print("Feature coefficients:")
for idx, row in coef_df_sorted.iterrows():
    feature = row['feature']
    coef_val = row['coefficient']
    direction = "positive" if coef_val > 0 else "negative"
    print(f"{feature:15s}: {coef_val:8.4f} ({direction})")
    if coef_val > 0:
        print(f"                 → Higher {feature} → Higher {TARGET}")
    else:
        print(f"                 → Higher {feature} → Lower {TARGET}")

# Simple visualization
plt.figure(figsize=(8, 5))
colors = ['red' if c < 0 else 'blue' for c in coef_df_sorted['coefficient']]
bars = plt.barh(coef_df_sorted['feature'], coef_df_sorted['coefficient'], color=colors, alpha=0.7)
plt.axvline(0, color='black', linestyle='-', linewidth=0.8)
plt.xlabel('Coefficient Value')
plt.title('Feature Coefficients\n(Red = Negative, Blue = Positive)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nMost important feature: {coef_df_sorted.iloc[0]['feature']}")
print(f"Largest coefficient: {coef_df_sorted.iloc[0]['coefficient']:.4f}")


## 10) Linear Regression — Quick Assumptions Checklist
- **Linearity**: Relationship between predictors and target is roughly linear.
- **Independence**: Errors are independent.
- **Homoscedasticity**: Residuals have constant variance.
- **Normality (for inference)**: Residuals are roughly normal.
- **No perfect multicollinearity**: Avoid duplicate/linearly dependent features.



## 11) Assumption Checks — Code You Can Run
We'll check the classic linear regression assumptions using simple, readable code:
- **Linearity & Homoscedasticity:** residuals vs predicted plot
- **Normality of residuals:** Q–Q plot and Shapiro–Wilk test
- **Independence of errors:** Durbin–Watson statistic
- **Multicollinearity:** Variance Inflation Factor (VIF)


In [ ]:

#@title Install test libs (statsmodels) if needed
import sys, subprocess
def pip_install(pkg):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=False)

try:
    import statsmodels
except:
    pip_install("statsmodels>=0.14")
    import statsmodels

from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_breuschpagan
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# NOTE: This cell assumes you've already run the multi-variable model section
# so that y_test, y_pred, X_train, X_test, numeric_cols, TARGET are defined.
# If not, re-run sections 6 and 7.


In [ ]:

# 11.1 Linearity & Homoscedasticity (visual)
# Residuals vs Predicted should look like a random cloud around 0 (no pattern / fanning)

residuals = y_test - y_pred

plt.figure()
plt.scatter(y_pred, residuals, s=14)
plt.axhline(0, linestyle="--")
plt.xlabel("Predicted")
plt.ylabel("Residuals")
plt.title("Residuals vs Predicted")
plt.show()

print("🧐 Look for: no obvious curve/pattern; spread roughly constant across x-axis.")


In [ ]:

# 11.2 Normality of residuals: Q–Q plot + Shapiro–Wilk test
plt.figure()
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q–Q Plot of Residuals")
plt.show()

sh_stat, sh_p = stats.shapiro(residuals)
print(f"Shapiro–Wilk: statistic={sh_stat:.3f}, p-value={sh_p:.3g}")
print("Rule of thumb: p-value > 0.05 suggests residuals are close to normal (for inference).")


In [ ]:

# 11.3 Independence of errors: Durbin–Watson (≈2 is good; <1 or >3 indicates strong autocorrelation)
dw = durbin_watson(residuals)
print(f"Durbin–Watson statistic = {dw:.3f}")
print("Guideline: ~2 means uncorrelated; much below 2 ⇒ positive autocorrelation; much above 2 ⇒ negative autocorrelation.")


In [ ]:

# 11.4 Homoscedasticity formal test: Breusch–Pagan
# Use original predictors (X_test). Add constant for the test design matrix.

X_bp = sm.add_constant(pd.DataFrame(X_test, columns=numeric_cols))
bp_stat, bp_p, _, _ = het_breuschpagan(residuals, X_bp)
print(f"Breusch–Pagan: stat={bp_stat:.3f}, p-value={bp_p:.3g}")
print("Rule of thumb: p-value > 0.05 ⇒ no strong evidence of heteroscedasticity.")


In [ ]:

# 11.5 Multicollinearity: VIF (on training predictors)
X_vif = pd.DataFrame(X_train, columns=numeric_cols).dropna()
X_vif_const = sm.add_constant(X_vif, has_constant='add')

vif_vals = []
for i, col in enumerate(X_vif_const.columns):
    if col == 'const':
        continue
    vif_vals.append({"feature": col, "VIF": variance_inflation_factor(X_vif_const.values, i+1)})

vif_df = pd.DataFrame(vif_vals).sort_values("VIF", ascending=False)
vif_df



### 📝 TODO
- If **Breusch–Pagan p < 0.05**, try transforming a skewed feature (e.g., `np.log1p(x)`) and re-fit.  
- If **Shapiro p < 0.05**, consider outliers or feature transforms.  
- If **VIF > 10** for a feature, try removing it or combining highly correlated features.  
- If **Durbin–Watson** is far from 2 (time-indexed data), try adding lag features or using time-series models.
